# Here We test <DB, Schema Independent Evaluators> 
- Test only with query, pred_sql, gold_sql
- Step1: Evaluate with only Negative data with existing dataset -> Calculate FP, TN
- Step2: Evaluate with full positive and negative data by adding hand-made poistive set -> Calculate TP ,FN  

### Make Negative Evaluation-pipeline test set from nl2sql_bug set
- Import nl2sql testest and create only-negative evaluation-pipeline test set 

In [ ]:
import pandas as pd
from azureml.fsspec import AzureMachineLearningFileSystem
from dotenv import load_dotenv
import os
import json

In [ ]:
load_dotenv()
data_storage_uri = os.getenv("AZURE_DATASTORAGE_URI")
fs = AzureMachineLearningFileSystem(data_storage_uri)
fs.ls()

In [ ]:
with fs.open('./UI/2025-07-25_003545_UTC/NL2SQL-Bugs-with-evidence.json') as f:
    data = json.load(f)

nl2sql_bug_df = pd.DataFrame(data)
nl2sql_bug_df["label"] = nl2sql_bug_df["label"].astype(bool)
len(nl2sql_bug_df.loc[nl2sql_bug_df["label"]== True])

In [ ]:
# 같은 Question은 공유하고 SQL은 다르게 적힌 True 있나 확인
dupes = (
    nl2sql_bug_df[nl2sql_bug_df["label"] == True]
    .groupby("question")
    #그룹 객체 대상으로 람다실행하고, 조건을 마족하는 원래 DataFrame의 row반환
    .filter(lambda x: len(x) > 1)
)
print(dupes) # 중복된 질문을 가진 True 라벨 row의 수
print(dupes["question"].unique())  # 어떤 question인지 보기



In [ ]:
# 0) Make sure your label column is really boolean
#    (if it's the strings "True"/"False" or ints 1/0, convert it)
nl2sql_bug_df["label"] = nl2sql_bug_df["label"].astype(bool)

# 1) Build a map from question → gold_sql (there must be at most one per question)
gold_map = (
    nl2sql_bug_df
    # 조건을 이용해 행선택하는 함수 
    .loc[nl2sql_bug_df["label"]]
    # 특정열을 인덱스로만 바꾸고, sql 만 선택  
    .set_index("question")["sql"]
    .to_dict()
)

len(gold_map)

In [ ]:
# 2) Walk through all FALSE‐label rows and emit one entry per pred_sql
eval_pipeline_test_list = []
for _, row in nl2sql_bug_df.loc[~nl2sql_bug_df["label"]].iterrows():
    q    = row["question"]
    pred = row["sql"]
    eval_pipeline_test_list.append({
        "question":    q,
        "db_id" : row["db_id"],
        "gold_sql":    gold_map.get(q),       # ← will be None only if no True‐row ever existed
        "pred_sql":    pred,
        "label":       False,
        "evidence":    row["evidence"],
        "error_types": row["error_types"],
    })



In [ ]:
eval_pipeline_test_df = pd.DataFrame(eval_pipeline_test_list)
eval_pipeline_test_df["gold_sql"].isnull().sum()
eval_pipeline_test_df = eval_pipeline_test_df.loc[eval_pipeline_test_df["gold_sql"].notnull()].copy()

In [ ]:
eval_pipeline_test_df

### Evaluate llm-as-judge-raw-sql-evaluators 
- Testing with only False dataset

In [ ]:
import sys
import os
from evaluators.llm_as_judge_raw_sql_evaluator import LLMasJudgeRawSQL
from azure.ai.evaluation import AzureOpenAIModelConfiguration
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

model_config_4o = AzureOpenAIModelConfiguration(
    azure_endpoint= os.environ["AZURE_ENDPOINT"],
    azure_key = os.environ["AZURE_API_KEY"],
    azure_deployment = os.environ["AZURE_4O_DEPLOYMENT"],
    api_version = os.environ["AZURE_4O_API_VERSION"]
)

model_config_o4_mini = AzureOpenAIModelConfiguration(
    azure_endpoint= os.environ["AZURE_ENDPOINT"],
    azure_key = os.environ["AZURE_API_KEY"],
    azure_deployment = os.environ["AZURE_O4_MINI_DEPLOYMENT"],
    api_version = os.environ["AZURE_O4_MINI_API_VERSION"]
)


In [ ]:
import json
from tqdm import tqdm

llm_sql_evaluator = LLMasJudgeRawSQL(model_config=model_config_o4_mini)
llm_raw_sql_pipeline_eval_result = []

for i, row in tqdm(eval_pipeline_test_df.iterrows(), ncols=100, colour="cyan", total=len(eval_pipeline_test_df)):
    question = row["question"]
    gold_sql = row["gold_sql"]
    pred_sql = row["pred_sql"]

    try:
        result = llm_sql_evaluator(question=question, gold_sql=gold_sql, pred_sql=pred_sql)
        result = json.loads(result)
        llm_raw_sql_pipeline_eval_result.append({
            "question": question,
            "gold_sql": gold_sql,
            "pred_sql": pred_sql,
            "llm_judgement": result["label"],
            "reason": result["reason"]
        })
        print(f"##### {i}th DEBUG LOG #####")
        print("Question:", question)
        print("Gold Sql:", gold_sql)
        print("Pred Sql:", pred_sql)
        print("LLM Judgement:", result["label"])
        print("Reason:", result["reason"])
    except json.JSONDecodeError as e:
        print(f"❌ JSONDecodeError at row {i}: {e}")
        print("⚠️ Raw response was:")
        print(repr(result))  # repr을 사용하면 문제되는 이스케이프 문자 확인 가능
        llm_raw_sql_pipeline_eval_result.append({
            "question": question,
            "gold_sql": gold_sql,
            "pred_sql": pred_sql,
            "llm_judgement": "ERROR",
            "reason": f"JSONDecodeError: {str(e)}"
        })
    except Exception as e:
        print(f"❌ Unexpected error at row {i}: {e}")
        llm_raw_sql_pipeline_eval_result.append({
            "question": question,
            "gold_sql": gold_sql,
            "pred_sql": pred_sql,
            "llm_judgement": "ERROR",
            "reason": f"Exception: {str(e)}"
        })




In [ ]:
llm_raw_sql_pipeline_eval_df = pd.DataFrame(llm_raw_sql_pipeline_eval_result)

#Join with Original Eval test set 

merged_llm_raw_sql_pipeline_eval_df = pd.merge(eval_pipeline_test_df, llm_raw_sql_pipeline_eval_df, on="pred_sql", how='left')
merged_llm_raw_sql_pipeline_eval_df.drop(["question_y","gold_sql_y"], axis = 1)

fp = len(merged_llm_raw_sql_pipeline_eval_df[merged_llm_raw_sql_pipeline_eval_df["llm_judgement"] == "correct"])
tn = len(merged_llm_raw_sql_pipeline_eval_df[merged_llm_raw_sql_pipeline_eval_df["llm_judgement"] == "incorrect"])


print("Total:", len(merged_llm_raw_sql_pipeline_eval_df))
print("FP:", fp)
print("TN:", tn)
print(fp/tn*100)


In [ ]:
print(merged_llm_raw_sql_pipeline_eval_df[merged_llm_raw_sql_pipeline_eval_df["llm_judgement"] == "correct"]["error_types"])

total_errortypes_count = {}
for _, row in merged_llm_raw_sql_pipeline_eval_df.iterrows():
    error_type = row["error_types"][0]["error_type"]
    sub_error_type = row["error_types"][0]["sub_error_type"]
    key_tuple = (error_type,sub_error_type)
    total_errortypes_count[key_tuple] = total_errortypes_count.get(key_tuple,0) +1

print(total_errortypes_count)

fp_errortypes_count = {}
for _, row in merged_llm_raw_sql_pipeline_eval_df[merged_llm_raw_sql_pipeline_eval_df["llm_judgement"] == "correct"].iterrows():
    error_type = row["error_types"][0]["error_type"]
    sub_error_type = row["error_types"][0]["sub_error_type"]
    key_tuple = (error_type,sub_error_type)
    fp_errortypes_count[key_tuple] = fp_errortypes_count.get(key_tuple,0) +1

print(fp_errortypes_count)




In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 두 딕셔너리의 모든 고유 키를 추출
all_keys = sorted(list(set(list(total_errortypes_count.keys()) + list(fp_errortypes_count.keys()))))

# 모든 키를 포함하고 누락된 키는 0으로 채운 완전한 딕셔너리 생성
full_dist1 = {key: total_errortypes_count.get(key, 0) for key in all_keys}
full_dist2 = {key: fp_errortypes_count.get(key, 0) for key in all_keys}

# DataFrame으로 변환 (시각화를 위해 긴 형식으로 변환)
df1 = pd.DataFrame(list(full_dist1.items()), columns=['Error Type', 'Count'])
df1['Distribution'] = 'Distribution 1'

df2 = pd.DataFrame(list(full_dist2.items()), columns=['Error Type', 'Count'])
df2['Distribution'] = 'Distribution 2'

# 두 데이터프레임을 하나로 합치기
combined_df = pd.concat([df1, df2])

# 오류 카테고리와 유형 분리 (튜플 형태의 키를 분리)
combined_df[['Error Category', 'Error Subtype']] = pd.DataFrame(combined_df['Error Type'].tolist(), index=combined_df.index)

# 시각화를 위한 정렬 (Count가 높은 순서대로)
# 모든 오류 유형에 대한 총 Count를 기준으로 정렬
error_order = combined_df.groupby('Error Subtype')['Count'].sum().sort_values(ascending=False).index

# 시각화 설정
plt.figure(figsize=(16, 10)) # 그래프 크기 설정
sns.set_theme(style="whitegrid") # 스타일 설정

# 막대 그래프 그리기 (오류 유형별로 Distribution 1과 Distribution 2 비교)
ax = sns.barplot(
    x='Count',
    y='Error Subtype',
    hue='Distribution',
    data=combined_df,
    order=error_order,
    palette='viridis' # 색상 팔레트 설정
)

plt.title('Comparison of Error Distributions', fontsize=16) # 그래프 제목
plt.xlabel('Number of Errors', fontsize=12) # x축 레이블
plt.ylabel('Error Type', fontsize=12) # y축 레이블
plt.legend(title='Distribution', fontsize=10, title_fontsize=12) # 범례

# y축 레이블 텍스트가 잘리지 않도록 레이아웃 조정
plt.tight_layout()

plt.show() # 그래프 출력

# ---
# 보너스: 상대 빈도 비교 시각화 (더 명확한 비율 비교)
# ---

# 상대 빈도 계산 (이전 코드에서 계산했던 방식 재사용)
total_dist1_count = df1['Count'].sum()
total_dist2_count = df2['Count'].sum()

df1['Relative Frequency (%)'] = (df1['Count'] / total_dist1_count) * 100
df2['Relative Frequency (%)'] = (df2['Count'] / total_dist2_count) * 100

combined_relative_df = pd.concat([df1, df2])

# 상대 빈도를 기준으로 정렬 (Distribution 1의 상대 빈도를 기준으로)
relative_error_order = combined_relative_df[combined_relative_df['Distribution'] == 'Distribution 1'] \
    .sort_values(by='Relative Frequency (%)', ascending=False)['Error Subtype']

plt.figure(figsize=(16, 10))
sns.barplot(
    x='Relative Frequency (%)',
    y='Error Subtype',
    hue='Distribution',
    data=combined_relative_df,
    order=relative_error_order,
    palette='magma' # 다른 색상 팔레트
)

plt.title('Comparison of Error Distributions by Relative Frequency', fontsize=16)
plt.xlabel('Relative Frequency (%)', fontsize=12)
plt.ylabel('Error Type', fontsize=12)
plt.legend(title='Distribution', fontsize=10, title_fontsize=12)
plt.tight_layout()
plt.show()

### Evaluate Execution Match with LLM as a Judge
- We will gonna implement Execution Match metrics and decide with LLM
- We will qeury to database and retrieve result from it, using our pipeline-evaluation set 

In [ ]:
import os 
import pandas as pd 
from db_utils.db_utils import get_db_path, execute_query, format_results_for_llm

#Base_dir should be path for project root
base_dir = os.path.abspath("..")

exec_match_test_df = eval_pipeline_test_df.copy()

exec_match_test_df["gold_result"] = None
exec_match_test_df["pred_result"] = None




In [ ]:
# 5) 순회하며 실행 및 포매팅
for idx, row in exec_match_test_df.iterrows():
    db_id    = row["db_id"]
    gold_sql = row["gold_sql"]
    pred_sql = row["pred_sql"]

    try:
        db_path = get_db_path(base_dir, db_id)
    except FileNotFoundError as e:
        exec_match_test_df.at[idx, "gold_result"] = f"ERROR: {e}"
        exec_match_test_df.at[idx, "pred_result"] = f"ERROR: {e}"
        continue

    # gold 실행 → 포매팅
    gold_raw = execute_query(db_path, gold_sql)
    exec_match_test_df.at[idx, "gold_result"] = format_results_for_llm(
        gold_raw,
        sort_keys=list(gold_raw[0].keys()) if gold_raw else None,
        row_limit=10
    )

    # pred 실행 → 포매팅
    pred_raw = execute_query(db_path, pred_sql)
    exec_match_test_df.at[idx, "pred_result"] = format_results_for_llm(
        pred_raw,
        sort_keys=list(pred_raw[0].keys()) if pred_raw else None,
        row_limit=5
    )


In [ ]:
from db_utils.db_utils import print_markdown_table

# 6) 결과 확인
exec_match_test_df[["db_id","gold_result","pred_result"]].head(10)


In [ ]:
for idx in range(3):
    print(f"\n====== ✅ Row {idx} / DB: {exec_match_test_df.iloc[idx]['db_id']} ======\n")
    
    print_markdown_table(
        md_str=exec_match_test_df.iloc[idx]["gold_result"],
        title="GOLD RESULT"
    )
    
    print_markdown_table(
        md_str=exec_match_test_df.iloc[idx]["pred_result"],
        title="PREDICTED RESULT"
    )


In [ ]:
from evaluators.llm_as_judge_exec_match_evaluator import LLMasJudgeExecMatch


llm_exec_match_evaluator = LLMasJudgeExecMatch(model_config=model_config_o4_mini)
llm_exec_match_pipeline_eval_result = []

for i, row in tqdm(exec_match_test_df.iterrows(), ncols=100, colour="cyan", total=len(eval_pipeline_test_df)):
    question = row["question"]
    gold_result = row["gold_result"]
    pred_result = row["pred_result"]

    try:
        result = llm_exec_match_evaluator(question=question, gold_result=gold_result, pred_result=pred_result)
        result = json.loads(result)
        llm_exec_match_pipeline_eval_result.append({
            "question": question,
            "gold_result": gold_result,
            "pred_result": pred_result,
            "llm_judgement": result["label"],
            "reason": result["reason"]
        })
        print(f"##### {i}th DEBUG LOG #####")
        print("Question:", question)
        print("Gold Result:", print_markdown_table(gold_result))
        print("Pred Result:", print_markdown_table(pred_result))
        print("LLM Judgement:", result["label"])
        print("Reason:", result["reason"])

    except json.JSONDecodeError as e:
        print(f"❌ JSONDecodeError at row {i}: {e}")
        print("⚠️ Raw response was:")
        print(repr(result))  # repr을 사용하면 문제되는 이스케이프 문자 확인 가능
        llm_raw_sql_pipeline_eval_result.append({
            "question": question,
            "gold_result": gold_result,
            "pred_result": pred_result,
            "llm_judgement": "ERROR",
            "reason": f"JSONDecodeError: {str(e)}"
        })
    except Exception as e:
        print(f"❌ Unexpected error at row {i}: {e}")
        llm_raw_sql_pipeline_eval_result.append({
            "question": question,
            "gold_result": gold_result,
            "pred_result": pred_result,
            "llm_judgement": "ERROR",
            "reason": f"Exception: {str(e)}"
        })




In [ ]:
llm_exec_match_pipeline_eval_df = pd.DataFrame(llm_exec_match_pipeline_eval_result)
fp = len(llm_exec_match_pipeline_eval_df[llm_exec_match_pipeline_eval_df["llm_judgement"] == "correct"])
tn = len(llm_exec_match_pipeline_eval_df[llm_exec_match_pipeline_eval_df["llm_judgement"] == "incorrect"])

print("Total:", len(llm_exec_match_pipeline_eval_df))
print("FP:", fp)
print("TN:", tn)
print(fp/tn*100)

In [ ]:
#Join with Original Eval test set 

merged_llm_exec_match_pipeline_eval_df = pd.merge(exec_match_test_df, llm_exec_match_pipeline_eval_df, on="pred_result", how='left')
merged_llm_exec_match_pipeline_eval_df.drop(["question_y","gold_result_y"], axis = 1)

In [ ]:
total_errortypes_count = {}
for _, row in merged_llm_exec_match_pipeline_eval_df.iterrows():
    error_types = [ x["error_type"] for x in row["error_types"]]
    sub_error_types = [ x["sub_error_type"] for x in row["error_types"]]
    key_tuples = zip(error_types,sub_error_types)
    for key_tuple in key_tuples:
        total_errortypes_count[key_tuple] = total_errortypes_count.get(key_tuple,0) +1

print(total_errortypes_count)

fp_errortypes_count = {}
for _, row in merged_llm_exec_match_pipeline_eval_df[merged_llm_exec_match_pipeline_eval_df["llm_judgement"] == "correct"].iterrows():
    error_types = [x["error_type"] for x in row["error_types"]]
    sub_error_types = [x["sub_error_type"] for x in row["error_types"]]
    key_tuples = zip(error_types, sub_error_types)
    for key_tuple in key_tuples:
        fp_errortypes_count[key_tuple] = fp_errortypes_count.get(key_tuple, 0) + 1



print(fp_errortypes_count)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ✅ 에러 딕셔너리 예시 (이 부분은 이미 가지고 있다고 가정)
# total_errortypes_count = {...}
# fp_errortypes_count = {...}

# ✅ 모든 고유 키 수집 및 누락된 키는 0으로 채우기
all_keys = sorted(set(total_errortypes_count.keys()).union(set(fp_errortypes_count.keys())))
full_dist1 = {key: total_errortypes_count.get(key, 0) for key in all_keys}
full_dist2 = {key: fp_errortypes_count.get(key, 0) for key in all_keys}

# ✅ 데이터프레임 생성
df1 = pd.DataFrame(list(full_dist1.items()), columns=['Error Type', 'Count'])
df1['Distribution'] = 'Distribution 1'

df2 = pd.DataFrame(list(full_dist2.items()), columns=['Error Type', 'Count'])
df2['Distribution'] = 'Distribution 2'

# ✅ Error Category, Error Subtype 컬럼 분리
df1[['Error Category', 'Error Subtype']] = pd.DataFrame(df1['Error Type'].tolist(), index=df1.index)
df2[['Error Category', 'Error Subtype']] = pd.DataFrame(df2['Error Type'].tolist(), index=df2.index)

# ✅ 병합
combined_df = pd.concat([df1, df2])

# ✅ 정렬 기준 설정
error_order = combined_df.groupby('Error Subtype')['Count'].sum().sort_values(ascending=False).index

# ✅ 절대 Count 기반 시각화
plt.figure(figsize=(16, 10))
sns.set_theme(style="whitegrid")
sns.barplot(
    x='Count',
    y='Error Subtype',
    hue='Distribution',
    data=combined_df,
    order=error_order,
    palette='viridis'
)
plt.title('Comparison of Error Distributions', fontsize=16)
plt.xlabel('Number of Errors', fontsize=12)
plt.ylabel('Error Subtype', fontsize=12)
plt.legend(title='Distribution', fontsize=10, title_fontsize=12)
plt.tight_layout()
plt.show()

# ✅ 상대 비율 계산
df1['Relative Frequency (%)'] = (df1['Count'] / df1['Count'].sum()) * 100
df2['Relative Frequency (%)'] = (df2['Count'] / df2['Count'].sum()) * 100

df1[['Error Category', 'Error Subtype']] = pd.DataFrame(df1['Error Type'].tolist(), index=df1.index)
df2[['Error Category', 'Error Subtype']] = pd.DataFrame(df2['Error Type'].tolist(), index=df2.index)

combined_relative_df = pd.concat([df1, df2])

# ✅ 상대 비율 기준 정렬
relative_error_order = combined_relative_df[combined_relative_df['Distribution'] == 'Distribution 1'] \
    .sort_values(by='Relative Frequency (%)', ascending=False)['Error Subtype']

# ✅ 상대 빈도 기반 시각화
plt.figure(figsize=(16, 10))
sns.barplot(
    x='Relative Frequency (%)',
    y='Error Subtype',
    hue='Distribution',
    data=combined_relative_df,
    order=relative_error_order,
    palette='magma'
)
plt.title('Comparison of Error Distributions by Relative Frequency', fontsize=16)
plt.xlabel('Relative Frequency (%)', fontsize=12)
plt.ylabel('Error Subtype', fontsize=12)
plt.legend(title='Distribution', fontsize=10, title_fontsize=12)
plt.tight_layout()
plt.show()
